### Overview

[**Black Friday Sales**](https://www.kaggle.com/c/gb-black-friday-sales)  
In this competition your task will be to predict the purchase amount in test.csv. You will be given two datasets: train.csv (contains all features and Purchase) and test.csv (only features).

### Data fields

* `User_ID` - идентификационный номер покупателя
* `Product_ID` - идентификационный номер товара
* `Gender` - пол покупателя 
* `Age` - возраст покупателя
* `Occupation` - род деятельности покупателя
* `City_Category` - город проживания
* `Stay_In_Current_City_Years` - как долго покупатель живет в этом городе
* `Marital_Status` - семейное положение покупателя
* `Product_Category_1` - категория товара 1
* `Product_Category_2` - категория товара 2
* `Product_Category_3` - категория товара 3

**Целевая переменная**

* `Purchase` - сумма покупок

**Метрика качества**

* `R2` - The evaluation metric is Coefficient of determination (sklearn.metrics.r2_score)

### File descriptions

* `train.csv` - the training set
* `test.csv` - the test set

### Подключение библиотек и загрузка данных

In [1]:
import numpy as np
import pandas as pd

In [58]:
def get_features(features):
    for feature in features:
        print(f"\n{feature}:\n \n{df_train[feature].value_counts()}" + '\n' + '*'*50)

In [2]:
TRAIN_DATASET_PATH='train.csv'
TEST_DATASET_PATH='test.csv'
PREPARED_TRAIN_DATASET_PATH='train_prepared.csv'
PREPARED_TEST_DATASET_PATH='test_prepared.csv'

In [3]:
df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_test = pd.read_csv(TEST_DATASET_PATH)

In [4]:
df_train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1004085,P00075742,F,26-35,6,A,1,0,8,14.0,NaN,7803
1,1005491,P00234842,M,18-25,7,A,1,0,5,6.0,16.0,6903
2,1003499,P00220142,M,26-35,3,A,2,0,1,15.0,NaN,15773
3,1000097,P00211242,F,36-45,3,C,3,0,8,12.0,NaN,8116
4,1005802,P00327142,F,26-35,0,A,4+,0,8,15.0,NaN,6144


### Обзор данных

In [5]:
df_train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,8.000000e+04,80000.000000,80000.000000,80000.000000,54892.000000,24331.000000,80000.000000
mean,1.003023e+06,8.086863,0.409388,5.388463,9.851873,12.676421,9243.933162
std,1.728959e+03,6.536459,0.491724,3.922793,5.079163,4.109343,5021.298356
min,1.000001e+06,0.000000,0.000000,1.000000,2.000000,3.000000,12.000000
25%,1.001503e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5679.750000
50%,1.003067e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8039.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12046.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     80000 non-null  int64  
 1   Product_ID                  80000 non-null  object 
 2   Gender                      80000 non-null  object 
 3   Age                         80000 non-null  object 
 4   Occupation                  80000 non-null  int64  
 5   City_Category               80000 non-null  object 
 6   Stay_In_Current_City_Years  80000 non-null  object 
 7   Marital_Status              80000 non-null  int64  
 8   Product_Category_1          80000 non-null  int64  
 9   Product_Category_2          54892 non-null  float64
 10  Product_Category_3          24331 non-null  float64
 11  Purchase                    80000 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 7.3+ MB


In [27]:
target = 'Purchase'
features = list(df_train.columns)[:-1]

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3']

* **Обзор целевой переменной**

* `Purchase`  - 80000 значений. среднее значение ~ 9244. Стандартное отклонение ~ 5021. Видно, что есть слишком большие и слишком маленькие значения. Нужно строить график распределения и делать оценку, исходя из других признаков 

In [28]:
purchase = df_train[target]
purchase.mean() - purchase.std(), purchase.mean() + purchase.std()

(4222.634806164755, 14265.231518835244)

In [49]:
purchase.value_counts()

7075     38
6949     37
7951     35
8032     35
6884     34
         ..
17149     1
21311     1
14088     1
9137      1
9686      1
Name: Purchase, Length: 13314, dtype: int64

* **Обзор признаков** 

In [59]:
get_features(features)


User_ID:
 
1004277    144
1001680    141
1001181    133
1001150    127
1001088    124
          ... 
1005199      1
1002709      1
1004920      1
1005637      1
1002287      1
Name: User_ID, Length: 5763, dtype: int64
**************************************************

Product_ID:
 
P00265242    272
P00025442    225
P00112142    224
P00058042    221
P00110742    220
            ... 
P00318442      1
P00247442      1
P00236242      1
P00188342      1
P00087942      1
Name: Product_ID, Length: 3256, dtype: int64
**************************************************

Gender:
 
M    60068
F    19932
Name: Gender, dtype: int64
**************************************************

Age:
 
26-35    32067
36-45    16025
18-25    14286
46-50     6777
51-55     5555
55+       3110
0-17      2180
Name: Age, dtype: int64
**************************************************

Occupation:
 
4     10443
0     10188
7      8474
1      6921
17     5815
20     4887
12     4559
14     4030
2      3849
16     372

* `User_ID` - идентификационный номер покупателя. Видно, что есть пользователи, которые делают много покупок, и есть такие, которые сделали только одну покупку. Можно выделить постоянных покупателей  
* `Product_ID` - идентификационный номер товара. Похожий признак, есть популярные товары, которые покупают чаще всего, а есть товары, которые покупают реже  
* `Gender` - пол покупателя. Мужчины делают покупки чаще, чем женщины. Возможно, какой-то специфический магазин (например, автозапчасти или что-то подобное)  
* `Age` - возраст покупателя. Видно, что пользователи разбиты на 7 групп, видно, что наиболее частые покупатели - люди от 26 до 35, и реже всего покупают дети до 17 лет
* `Occupation` - род деятельности покупателя - признак состоит из большого количества категорий, чаще всего встречаются категории 4, 0, 7, реже всего - 18, 9, 8   
* `City_Category` - категория города (скорее всего, разбитые по численности населения). Самая популярная категория - "В"  
* `Stay_In_Current_City_Years` - как долго покупатель живет в этом городе. Чаще всего покупают люди, которые живут в городе 1 год
* `Marital_Status` - семейное положение покупателя. Больше всего значений в категории 0

Скорее всего, в магазине товары разбиты на 3 категории, в котором есть много подкатегорий
* `Product_Category_1` - категория товара 1   
* `Product_Category_2` - категория товара 2  - Очень много нулей (больше половины) 
* `Product_Category_3` - категория товара 3 - Тоже очень много нулей